# Python Web Scraping Tutorial #2: ใช้ Requests มาช่วย Pandas ดึงข้อมูล

by Nior Atthakorn

จาก[ตอนที่แล้ว](https://medium.com/@nioratthakorn/python-web-scraping-tutorial-1-9cba93ac2690) เราได้ใช้ [Pandas](https://pandas.pydata.org) ในการทำ Web Scraping มาแล้ว ซึ่งก็มีข้อจำกัดอยู่หลายอย่าง ไม่ว่าจะเป็นการที่ข้อมูลจะต้องเป็นตารางเท่านั้น หรือตัว url ที่จะมีตัวอักษรที่แปลกประหลาดหน่อย (อย่างเช่นตัวอักษรภาษาไทย) ไม่ได้เลย

ในบทความนี้เราจึงจะใช้ [Requests](https://pypi.org/project/requests/) ซึ่งเป็น library สำหรับการส่ง http request อย่างง่าย มาช่วยแก้ปัญหาที่ตัว Pandas ไม่สามารถอ่าน url แปลกๆ ได้ จะมีวิธีการทำยังไง ก็อ่านต่อด้านล่างนี้เลย Let's Go!

<table align="center">
    <td>
        <a href="https://github.com/NiorAP/web_scraping_tutorials/blob/master/Python%20Web%20Scraping%20Tutorial%20%232/Python%20Web%20Scraping%20Tutorial%20%232.ipynb">
            <img src="https://pngimg.com/uploads/github/github_PNG58.png" width="100"/>
            <center>
                ดูใน GitHub
            </center>
        </a>
    </td>
    <td>
        <a href="https://colab.research.google.com/drive/1ymFoRjC8Fv4O4r8hq90az81weJCH9DA_">
            <img src="https://colab.research.google.com/img/colab_favicon_256px.png" width="100"/>
            <center>
                ลองเล่นใน Google Colab
            </center>
        </a>
    </td>
    <td>
        <a href="https://medium.com/@nioratthakorn/python-web-scraping-tutorial-2-7a8d09a36093">
            <img src="https://cdn4.iconfinder.com/data/icons/vector-brand-logos/40/Medium-512.png"/ width="100">
            <center>
                อ่านใน Medium
            </center>
        </a>
    </td>
</table>

[ก่อนหน้านี้](https://medium.com/@nioratthakorn/python-web-scraping-tutorial-1-9cba93ac2690)เราใช้แค่ function [read_html](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.read_html.html) ของ Pandas โดยใส่ url เข้าไป ก็สามารถทำการ Scrape ข้อมูลที่เราต้องการได้ทันที แต่ถ้า url ใช้ไม่ได้หล่ะ ?

ถ้ายังจำกันได้[จากขั้นตอนที่ 3](https://medium.com/@nioratthakorn/python-web-scraping-tutorial-1-9cba93ac2690#b5fb) ใน[บทความที่แล้ว](https://medium.com/@nioratthakorn/python-web-scraping-tutorial-1-9cba93ac2690) ที่บอกไว้ว่าตัว `read_html` สามารถรับ parameter ได้หลายรูปแบบ โดยในบทความนี้เราจะส่ง HTML text ให้ `read_html` ดึงข้อมูลที่เป็นตารางที่เราต้องการจากหน้า website

แล้วเราจะได้ HTML text มาได้ยังไง ? ก็แหงหล่ะครับ ดูจากชื่อบทความก็น่าจะพอเดาได้ไม่ยากนะครับ เราจะใช้ library ที่ชื่อ Requests มาดึงข้อมูล HTML จาก website นั่นเองงงง

<i><center><b>ในเมื่อ Pandas ดึงข้อมูลจาก website ที่มี url แปลกๆ ไม่ได้ ก็ให้ Requests ดึงมาให้ แล้วให้ Pandas จัดการต่อ</b></center></i>

<i><b>ขั้นตอนที่ 1</b></i> ก็เหมือนๆ เดิมครับ `import` library ที่จะใช้ ในที่นี้จะใช้ 2 ตัว
- Requests -> ดึงข้อมูลหน้า website
- Pandas -> แปลงข้อมูลที่ได้มาเป็นตาราง

In [1]:
import requests
import pandas as pd

<i><b>ขั้นตอนที่ 2</b></i> ใส่ url ของ website ที่เราต้องการ scrape เหมือนเดิม ซึ่งจาก[บทความแรก](https://medium.com/@nioratthakorn/python-web-scraping-tutorial-1-9cba93ac2690) url ต้องเป็นภาษาอังกฤษเท่านั้น (ถ้า url มีภาษาไทย ใช้วิธีเดิมไม่ได้) แต่ในบทความนี้สามารถใช้ได้ ในที่นี้จะใช้ website [สถิติหวยย้อนหลัง 29 ปี](https://www.myhora.com/หวย/สถิติหวย-ย้อนหลัง-25-ปี.aspx?mode=year-range&value=29) ของ [myhora.com](https://www.myhora.com/) ต้องขออนุญาตและขอขอบคุณมา ณ ที่นี้ด้วยนะครับ -/\\-

In [2]:
url = 'https://www.myhora.com/หวย/' \
    + 'สถิติหวย-ย้อนหลัง-29-ปี.aspx?mode=year-range&value=29'

url ยาว เลยเขียนแยกเป็น 2 ท่อนเหมือนเดิมจ้า

<i><b>ขั้นตอนที่ 3</b></i> ให้เราใช้ function [`get`](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) ของ Requests ในการดึงข้อมูลหน้า website จาก url ที่เราใส่

In [3]:
response = requests.get(url)

<i><b>ขั้นตอนที่ 4</b></i> ให้เราใช้ function `read_html` ของ pandas โดยตัวอย่างที่แล้วเราใส่ url เป็น argument ซึ่งจะทำให้ตัว function ไปดึงข้อมูลทั้งหมดจาก website นั้นๆ แต่ในที่นี้เราใส่ข้อมูล HTML text เข้าไปเลย ตัว function `read_html` ก็จะ scan ดูทั้ง HTML ที่ใส่เข้าไป ว่ามีตารางอยู่ตรงไหนบ้าง แปลงแต่ละตารางเป็น [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) แล้วใส่ไว้ใน list ให้เราได้เหมือนเดิม สะดวกเกิ๊น

In [4]:
dfs = pd.read_html(response.text)

ตอนนี้ก็จบขั้นตอนการ scrape แล้ว มาลองดูผลลัพธ์กันสักหน่อย

ลองดูก่อนเหมือนเดิมว่าจำนวนตารางที่ได้ตรงกับในหน้า website ที่เราเห็นมั้ย

In [5]:
len(dfs)

29

เจอตารางมากถึง 29 ตาราง แต่เอ๊ะ! ลองเลื่อนดูผ่านๆ ในหน้า website มันก็ดูไม่น่ามีถึง 29 ตารางนี่นา ทำไมมันออกมาตั้ง 29 ตาราง ?

ถ้าคุณคิดแบบนี้ไม่ต้องตกใจไปครับ (เพราะตอนแรกผมก็คิดแบบนั้น) ในหน้า website เขามีบางตารางที่ (เหมือนกับ) ซ่อนอยู่ (แต่จริงๆ เขาไม่ได้ซ่อนน้า) ลองเลื่อนไปดูแถวๆ ด้านล่างของแต่ละกล่อง จะเจอปุ่มประมาณนี้

![](https://github.com/NiorAP/web_scraping_tutorials/raw/master/Python%20Web%20Scraping%20Tutorial%20%232/Image-2-1.png)

ลองกดดูซิ จะเป็นยังไงน้าาาา

![](https://github.com/NiorAP/web_scraping_tutorials/raw/master/Python%20Web%20Scraping%20Tutorial%20%232/Image-2-2.png)

นี่ไง มีตารางซ่อนอยู่ในนี้อีก ซึ่งถ้าเราลองกดดูทั้งหมดทั้งมวล แล้วลองนับตารางดู เราก็จะได้ 29 ตารางพอดิบพอดีเลย

เนื่องจากใน website นี้มีตารางเยอะมาก ในที่นี้เลยขอดึงมาดูแค่ 2 ตาราง คือตารางแรกกับตารางสุดท้าย ว่าได้หน้าตาอย่างที่เราอยากได้มั้ย

เริ่มจากตารางแรกก่อนเลย

In [6]:
dfs[0].head()

,0,1,2,3,4,5,6,7,8,9,10
0,หลัก,เลข0,เลข1,เลข2,เลข3,เลข4,เลข5,เลข6,เลข7,เลข8,เลข9
1,สิบ,68,72,71,62,74,78,72,71,66,62
2,หน่วย,65,78,68,72,94,61,66,88,64,40
3,รวม,133,150,139,134,168,139,138,159,130,102


พอได้อยู่ อาจจะต้องจัดการอีกนิดนึง แต่ก็ถือว่าใช้ได้แล้ว

มาลองดูตารางสุดท้ายกันบ้าง

In [7]:
dfs[-1].head()

,0
0,16 มีนาคม มี.ค. 2563 63 503446 46 446 77 258 ...
1,1 มีนาคม มี.ค. 2563 63 875938 38 938 98 294 3...
2,16 กุมภาพันธ์ ก.พ. 2563 63 781403 03 403 94 5...
3,1 กุมภาพันธ์ ก.พ. 2563 63 589227 27 227 06 25...
4,17 มกราคม ม.ค. 2563 63 491774 74 774 68 004 1...


<i>อ้าวเฮ้ย ไม่เหมือนที่คุยกันไว้นี่หน่า</i>

จากด้านบนจะเห็นว่า เราได้ตารางที่เราอยากได้แล้ว แต่อยู่ในรูปแบบที่เรายังไม่ต้องการ ข้อมูลทั้งหมดอยู่ใน column เดียวกัน เราต้องการ split ออกจากกัน

ในส่วนนี้เป็นการใช้งาน Pandas ในการจัดการข้อมูลที่มี ไม่เกี่ยวข้องกับการ scrape แล้ว แต่ก็มาลองทำกันดูสักหน่อยจะเป็นไรไป เนอะๆๆๆๆ

In [8]:
df = dfs[-1][0].str.split(expand=True)

โอ้โห อะไรไม่รู้ยั้วเยี๊ยเต็มไปหมดเลย ลองค่อยๆ เจาะดูกันทีละส่วนครับ

```python
dfs[-1]
```
ส่วนแรกนี้คือการเลือกตารางสุดท้ายออกมาจากผลลัพธ์การ scrape ซึ่งก็คือตารางสุดยุ่งเหยิงที่เราจะเอามาทำการแปลงโฉมนั่นเอง (โดยตารางเป็นประเภท Pandas DataFrame)

```python
dfs[-1][0]
```
หลังจากเลือกตารางแล้ว ก็มาเลือก column อีกที เนื่องจากคำสั่งที่เราจะใช้ต่อไปนี้เป็นคำสั่งที่ใช้กับ [Pandas Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) เท่านั้น

```python
dfs[-1][0].str
```
ตัว [`.str`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) นี้ทำให้เราสามารถใช้ method ที่คล้ายๆ ว่าจะเป็น [string method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.htmls) กับ Pandas Series ได้

```python
dfs[-1][0].str.split(expand=True)
```
ส่วนสุดท้ายนี้ก็คือ method [`split`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html) ที่จะตัดแบ่ง string ตามตัวอักษรที่เรากำหนดไว้ (ในที่นี้ไม่ได้กำหนดไว้ method ก็จะไปดึงค่า default  นั่นคือ space " " มาใช้ในการแบ่ง) พร้อมทั้งใส่ parameter `expand=True` เป็นการบอกให้กระจายผลลัพธ์ที่ได้จากการแบ่งเป็น column หลายๆ อัน

ดูสภาพหลังแปลงแล้ว

In [9]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,16,มีนาคม,มี.ค.,2563,63,503446,46,446,77,258,726,404,661,77,258,726,404,661
1,1,มีนาคม,มี.ค.,2563,63,875938,38,938,98,294,328,597,780,98,294,328,597,780
2,16,กุมภาพันธ์,ก.พ.,2563,63,781403,03,403,94,515,952,030,918,94,515,952,030,918
3,1,กุมภาพันธ์,ก.พ.,2563,63,589227,27,227,06,259,552,375,927,06,259,552,375,927
4,17,มกราคม,ม.ค.,2563,63,491774,74,774,68,004,132,379,595,68,004,132,379,595


สวยงามตามท้องเรื่องงงง

เป็นไงกันบ้างครับกับการใช้ Requests มาช่วย Pandas ทำ Web Scraping ง่ายเหมือนเดิมใช่ไหมหล่ะครับ แต่ก็ยังมีเงื่อนไขในการใช้งานอยู่อีกพอสมควร

ด้วยข้อจำกัดใหญ่ที่ข้อมูลต้องเป็นตารางเท่านั้น แล้วถ้าข้อมูลไม่ใช่ตารางหล่ะ ? เราจะทำยังไงต่อ ใช้เครื่องมืออะไร ติดตามได้ในตอนต่อๆ ไปนะครับ

ส่วนใครยังไม่ได้อ่านตอนแรก หรืออ่านแล้วลืมแล้วสามารถย้อนอ่านตอนแรกได้ที่นี่เลย:  
[Python Web Scraping Tutorial #1: ใช้ Pandas ดึงข้อมูลที่เป็นตารางจากหน้า Website](https://medium.com/@nioratthakorn/python-web-scraping-tutorial-1-9cba93ac2690)

สำหรับวันนี้ก็ แค่นี้แหละครับ บายยยย